# Lab 5 - Linear models and Regularization
- **Author:** Qutub Khan Vajihi ([qutubkhan.vajihi@berkeley.edu](mailto:qutubkhan.vajihi@berkeley.edu))
- **Update:** 24 Feb 2021
- **Course:** INFO 251: Applied Machine Learning

### Topics:
1. Lasso vs. Ridge
2. Cross-validation to find optimal regularization parameter

### References: 
 * [Lasso](http://scikit-learn.org/stable/modules/generated/sklearn.linear_model.Lasso.html#sklearn.linear_model.Lasso)
 * [Ridge](http://scikit-learn.org/stable/modules/generated/sklearn.linear_model.Ridge.html#sklearn.linear_model.Ridge)
 * [RidgeCV](http://scikit-learn.org/stable/modules/generated/sklearn.linear_model.RidgeCV.html#sklearn.linear_model.RidgeCV)
 * [cross_val_score](http://scikit-learn.org/stable/modules/generated/sklearn.model_selection.cross_val_score.html#sklearn.model_selection.cross_val_score)
 * [GridSearchCV](http://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html)

In [76]:
import pandas as pd
import numpy as np
import time
import matplotlib.pyplot as plt
import statsmodels.api as sm
import statsmodels.formula.api as smf
%matplotlib inline

For this lab, let's keep using the [Boston Housing Prices Data Set](http://www.kellogg.northwestern.edu/faculty/weber/emp/_session_3/boston.htm).

In [77]:
from sklearn.datasets import load_boston
bdata = load_boston()

In [78]:
boston = pd.DataFrame(bdata.data)
boston.columns = bdata.feature_names[:]
boston['MEDV'] = bdata.target

In [79]:
boston.head(3)

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,MEDV
0,0.00632,18.0,2.31,0.0,0.538,6.575,65.2,4.0900,1.0,296.0,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0.0,0.469,6.421,78.9,4.9671,2.0,242.0,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0.0,0.469,7.185,61.1,4.9671,2.0,242.0,17.8,392.83,4.03,34.7


In [80]:
set(boston.columns.tolist()) - set(['MEDV'])  #Set function

{'AGE',
 'B',
 'CHAS',
 'CRIM',
 'DIS',
 'INDUS',
 'LSTAT',
 'NOX',
 'PTRATIO',
 'RAD',
 'RM',
 'TAX',
 'ZN'}

In [81]:
v = list(set(boston.columns.tolist()) - set(['MEDV'])) #Like A intersection B complement
v_formular = '+'.join(v) 
v_formular #Format required for input features in R language

'B+NOX+AGE+LSTAT+CHAS+RM+PTRATIO+RAD+DIS+CRIM+INDUS+TAX+ZN'

In [82]:
model_1 = smf.ols(formula = 'MEDV ~ '+v_formular, data = boston).fit()
print(model_1.summary())

                            OLS Regression Results                            
Dep. Variable:                   MEDV   R-squared:                       0.741
Model:                            OLS   Adj. R-squared:                  0.734
Method:                 Least Squares   F-statistic:                     108.1
Date:                Wed, 24 Feb 2021   Prob (F-statistic):          6.72e-135
Time:                        12:47:33   Log-Likelihood:                -1498.8
No. Observations:                 506   AIC:                             3026.
Df Residuals:                     492   BIC:                             3085.
Df Model:                          13                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     36.4595      5.103      7.144      0.0

In [83]:
from sklearn import linear_model
reg = linear_model.LinearRegression(fit_intercept=True, normalize=False)
reg.fit(boston[v], boston['MEDV'])

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [84]:
pd.DataFrame({'Variable':v,'Coef':reg.coef_})

,Variable,Coef
0,B,0.009312
1,NOX,-17.766611
2,AGE,0.000692
3,LSTAT,-0.524758
4,CHAS,2.686734
5,RM,3.809865
6,PTRATIO,-0.952747
7,RAD,0.306049
8,DIS,-1.475567
9,CRIM,-0.108011


In [85]:
reg.score(boston[v], boston['MEDV'])

0.7406426641094094

In [86]:
pd.DataFrame({'variable':v, 'coefficient':reg.coef_})

,variable,coefficient
0,B,0.009312
1,NOX,-17.766611
2,AGE,0.000692
3,LSTAT,-0.524758
4,CHAS,2.686734
5,RM,3.809865
6,PTRATIO,-0.952747
7,RAD,0.306049
8,DIS,-1.475567
9,CRIM,-0.108011


### Oops! We didn't split the data into train and test.

In [87]:
from sklearn.model_selection import train_test_split

In [88]:
X_train, X_test, y_train, y_test = train_test_split(boston[v], boston['MEDV'], test_size=0.2, random_state=4973)
reg = linear_model.LinearRegression(fit_intercept=True, normalize=False)
reg.fit(X_train, y_train)
print("R square of testing set: {:.2f}".format(reg.score(X_test, y_test)))

R square of testing set: 0.67


# LASSO

Recall that the cost function is:

$ J(\theta) =  \frac{1}{2n}||Y - X\theta||^2_2 + \lambda ||\theta||_1$

In [89]:
clf = linear_model.Lasso(alpha=0.5)
clf.fit(X_train, y_train)
print("R square of testing set: {:.2f}".format(clf.score(X_test, y_test)))
# predict
y_test_predict = clf.predict(X_test)

R square of testing set: 0.64


In [90]:
print(clf.coef_)

[ 7.45462694e-03 -0.00000000e+00  1.35524520e-03 -6.02807768e-01
  0.00000000e+00  2.23368987e+00 -8.17468375e-01  2.54790191e-01
 -1.00239460e+00 -9.35220773e-02 -2.49928121e-02 -1.63628096e-02
  5.13102681e-02]


In [91]:
clf = linear_model.Lasso(alpha=10)
clf.fit(X_train, y_train)
print("R square of testing set: {:.2f}".format(clf.score(X_test, y_test)))
# predict
y_test_predict = clf.predict(X_test)

R square of testing set: 0.42


In [92]:
print(clf.coef_)

[ 0.00598737  0.          0.         -0.48795095  0.          0.
 -0.          0.         -0.         -0.         -0.         -0.01282478
  0.02977285]


Notice how so many coefficients are driven down to zero, as we increase the alpha(lambda).

# Ridge

Recall that the cost function is:

$ J(\theta) =  \frac{1}{2n}||Y - X\theta||^2_2 + \lambda \theta ^ 2$

In [93]:
reg = linear_model.Ridge(alpha = 0) #alpha is lambda
reg.fit(X_train, y_train)
print("R square of testing set: {:.2f}".format(reg.score(X_test, y_test)))

R square of testing set: 0.67


In [94]:
reg = linear_model.Ridge(alpha = 0.5)
reg.fit(X_train, y_train)
print("R square of testing set: {:.2f}".format(reg.score(X_test, y_test)))
print(reg.coef_)

R square of testing set: 0.66
[ 7.36824413e-03 -1.04643390e+01 -1.00567958e-02 -4.78587366e-01
  3.11727229e+00  3.60639639e+00 -9.05828481e-01  2.46907025e-01
 -1.43421303e+00 -1.08748277e-01 -4.29302231e-02 -1.27348052e-02
  4.68085482e-02]


In [95]:
reg = linear_model.Ridge(alpha = 1)
reg.fit(X_train, y_train)
print("R square of testing set: {:.2f}".format(reg.score(X_test, y_test)))
print(reg.coef_)

R square of testing set: 0.66
[ 7.45838010e-03 -7.99944788e+00 -1.20375073e-02 -4.81783601e-01
  3.07118379e+00  3.60513755e+00 -8.85582916e-01  2.40396767e-01
 -1.39475596e+00 -1.07614042e-01 -5.24476209e-02 -1.29042178e-02
  4.71547111e-02]


In [96]:
reg = linear_model.Ridge(alpha = 1000)
reg.fit(X_train, y_train)
print("R square of testing set: {:.2f}".format(reg.score(X_test, y_test)))
print(reg.coef_)

R square of testing set: 0.59
[ 0.00625535 -0.00525122  0.03191989 -0.69625069  0.10573254  0.45590949
 -0.64112413  0.28440211 -0.55952796 -0.09733338 -0.04101698 -0.01810787
  0.05606228]


Again, we should not be tuning our hyperparameter on the test data. Cross-validation is the correct way for hyperparameter tuning.

# Cross Validation - Tuning Lambda

In [97]:
al = list(np.array([1000,100,10,1,0.1,0.01,0.001,0.0001]))
al

[1000.0, 100.0, 10.0, 1.0, 0.1, 0.01, 0.001, 0.0001]

In [98]:
from sklearn.model_selection import cross_val_score
for i in al : 
    reg = linear_model.Ridge(alpha=i)
    scores = cross_val_score(reg, X_train, y_train, cv=50, scoring='r2')
    print(i)
    print('mean r square: {:.2f}'.format(np.mean(scores)))


1000.0
mean r square: 0.43
100.0
mean r square: 0.46
10.0
mean r square: 0.45
1.0
mean r square: 0.45
0.1
mean r square: 0.45
0.01
mean r square: 0.45
0.001
mean r square: 0.45
0.0001
mean r square: 0.45


### Let's also try out Grid Search

<img src="GridSearch.png" width=800 height=800 />

In [99]:
#Link: https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html
from sklearn.model_selection import GridSearchCV
alphas = np.array(np.array([1000,100,10,1,0.1,0.01,0.001,0.0001]))
model = linear_model.Ridge()
# param_grid: 
# Dictionary with parameters names (string) as keys and lists of parameter settings to try as values
grid = GridSearchCV(estimator=model, param_grid=dict(alpha=alphas), cv=50, scoring='r2') #Same as ridge cv but very useful when we need to tune multiple hyperparameters
grid.fit(X_train, y_train)
print(grid.best_score_)
print(grid.best_params_)

0.4592743292720323
{'alpha': 100.0}


You can use Grid Search for tuning more than one hyperparameter as well. 